In [1]:
import pandas as pd
import numpy as np

In [45]:
import warnings
warnings.filterwarnings('ignore')

#### OPTIMIZER

In [2]:
df_sety = pd.read_excel('CARDS.xlsx', sheet_name='SET')

df_karty = pd.read_excel('CARDS.xlsx', sheet_name='KARTY', usecols=['KARTA', 'TYP', 'DAMAGE', 'N_AWAKENING'])

df_karty = df_karty[df_karty['DAMAGE'] == 'DEMON']

koszt = pd.read_excel('CARDS.xlsx', sheet_name='KOSZT')

In [48]:
def oblicz(df_sety, I_SET, SET, df_karty):

    df1 = pd.DataFrame(columns=['POZIOM', 'KOSZT'])
    df2 = pd.DataFrame(columns=['N', 'N%', 'SET', 'KOSZT'])
    
    df_temp_set = df_sety[df_sety['SET'] == SET] 
    THRESHOLD = df_temp_set.loc[I_SET, ['N1', 'N2', 'N3']].to_list()
    THRESHOLD_DAMAGE = df_temp_set.loc[I_SET, ['N%1', 'N%2', 'N%3']].to_list()
    KARTY = df_sety.loc[I_SET,  'C1':'C10'].reset_index().rename(columns={I_SET: 'KARTA'}).merge(df_karty, 'left', on='KARTA').dropna(subset=['KARTA'])
    SUMA_KART = int(KARTY['N_AWAKENING'].sum())
    # print(KARTY)
    # print(SUMA_KART)

    for I_COLOR, kolor in enumerate(['Uncommon', 'Common', 'Rare', 'Epic', 'Legendary']):
        df_temp_kolor = KARTY[KARTY['TYP'] == kolor]
        if not df_temp_kolor.empty:
            for x, ix in zip(df_temp_kolor['N_AWAKENING'].astype(int).values, df_temp_kolor['KARTA'].values):
                df_temp=koszt[x:][kolor].reset_index().rename(columns={'index': 'POZIOM', kolor: 'KOSZT'}).assign(POZIOM=lambda x: x['POZIOM']+1).assign(SET=SET).assign(TYP=kolor).assign(KARTA=ix)
                df1 = pd.concat([df1, df_temp], ignore_index=True)
        else:
            pass
    df1 = df1.sort_values(by='KOSZT').reset_index(drop=True)
    # print(SET)
    # print(SUMA_KART)
    for T, T_DMG in zip(THRESHOLD, THRESHOLD_DAMAGE):
        if T <= SUMA_KART:
            pass
        else:
            try:
                # print(f'PUNKT: {T}, {T_DMG}')
                # print(df1['KOSZT'][:T-SUMA_KART].sum() - A)
                df2 = pd.concat([df2, pd.DataFrame(columns=['N', 'N%', 'SET', 'KOSZT'], data=[[T, T_DMG, SET, df1['KOSZT'][:T-SUMA_KART].sum() - A]])], ignore_index=True)
            except:
                # print(f'PUNKT: {N}')
                # print(df1['KOSZT'][:T-SUMA_KART].sum())
                df2 = pd.concat([df2, pd.DataFrame(columns=['N', 'N%', 'SET', 'KOSZT'], data=[[T, T_DMG, SET, df1['KOSZT'][:T-SUMA_KART].sum()]])], ignore_index=True)
            # print(f'PUNKT: {N}')
            A = df1['KOSZT'][:T-SUMA_KART].sum()
            # print(df['KOSZT'][:N-SUMA_KART].sum())
                
    return df1, df2           


df_all = pd.DataFrame()
df_all_set = pd.DataFrame()
for I_SET, SET in enumerate(df_sety['SET']):
    
    data1, data2 = oblicz(df_sety, I_SET, SET, df_karty)
    df_all = pd.concat([df_all, data1], ignore_index=True)
    df_all_set = pd.concat([df_all_set, data2], ignore_index=True)

In [51]:
(df_all_set.assign(KOSZT_JEDN=df_all_set['KOSZT']/(df_all_set['N%']*10000))
 .sort_values(by="KOSZT_JEDN")
 .astype({'KOSZT_JEDN': 'float'})
 .round({'KOSZT_JEDN': 2}))

,N,N%,SET,KOSZT,KOSZT_JEDN
1,35,0.0014,Light of Salvation,291000,20785.71
6,24,0.0010,Darkness Legion,247000,24700.00
20,10,0.0010,I Missed You,291000,29100.00
16,15,0.0010,Mystracia's Secret,291000,29100.00
3,24,0.0010,Ghost Busters,316000,31600.00
17,12,0.0010,A Voice Calls,378000,37800.00
18,15,0.0010,A Voice Calls,388000,38800.00
14,16,0.0007,Mormot,280000,40000.00
19,10,0.0007,The Lord of Mayhem,291000,41571.43
5,30,0.0014,The Ruins of Xeneela,615000,43928.57
